# Week 7 Homework

***Due (pushed to your GitHub branch) on 11/1 by 11:59 pm***

## Samtools

Use Samtools to convert to BAM and sort the output SAMs produced by HISAT:

In [1]:
!for sam in ../6_alignment/placenta_alignment/hisat/*.sam; do \
    bn=$(basename "$sam" .sam); \
    echo "Converting $sam to $bn.bam and sorting."; \
    samtools view -@ 32 -b $sam | samtools sort -@ 32 > ../6_alignment/placenta_alignment/hisat/$bn.bam; \
    done

Converting ../6_alignment/placenta_alignment/hisat/*.sam to *.bam and sorting.
[E::hts_open_format] Failed to open file ../6_alignment/placenta_alignment/hisat/*.sam
samtools view: failed to open "../6_alignment/placenta_alignment/hisat/*.sam" for reading: No such file or directory


Merge the BAMs to a single BAM and generate an index:

In [ ]:
!samtools merge -@ 32 ../6_alignment/placenta_alignment/hisat/merged.bam  ../6_alignment/placenta_alignment/hisat/*.bam 
!samtools index -@ 32 ../6_alignment/placenta_alignment/hisat/merged.bam 

Use Samtools `flagstat` to generate QC metrics for the STAR and HISAT merged BAMs:

In [ ]:
!samtools flagstat ../6_alignment/placenta_alignment/

## Qualimap

Use Qualimap to run `bamqc` and `rnaseq` to generate QC metrics for the STAR and HISAT merged BAMs:

In [ ]:
!qualimap bamqc -nt 32 -outdir qualimap/star/bam -bam ../6_alignment/placenta_alignment/star/Aligned.sortedByCoord.out.bam --feature-file ../2_genome_exploration/placenta_genome/placenta_annotations.gtf -p strand-specific-forward
!qualimap rnaseq -outdir qualimap/star/rnaseq -bam ../6_alignment/placenta_alignment/star/Aligned.sortedByCoord.out.bam -gtf ../2_genome_exploration/placenta_genome/placenta_annotations.gtf -p strand-specific-forward
!qualimap bamqc -nt 32 -outdir qualimap/hisat/bam -bam ../6_alignment/placenta_alignment/hisat/merged.bam --feature-file ../2_genome_exploration/placenta_genome/placenta_annotations.gtf -p strand-specific-forward
!qualimap rnaseq -outdir qualimap/hisat/rnaseq -bam ../6_alignment/placenta_alignment/hisat/merged.bam -gtf ../2_genome_exploration/placenta_genome/placenta_annotations.gtf -p strand-specific-forward
# !echo "HISAT mapping stats"
# !samtools stats  -@ 32 ../6_alignment/placenta_alignment/hisat/merged.bam > placenta_hisat_stats.txt
# !echo "STAR index"
# !samtools index -@ 32 ../6_alignment/placenta_alignment/star/Aligned.sortedByCoord.out.bam
# !echo "STAR mapping stats"
# !samtools stats -@ 32 ../6_alignment/placenta_alignment/star/Aligned.sortedByCoord.out.bam > placenta_star_stats.txt

## Report

Use MuliQC to generate a single report that merges all Samtools and Qualimap metrics:

In [ ]:
!multiqc --force -d ../5_fastq/fastq_placenta/placenta_qc/ ../5_fastq/fastq_placenta/placenta_qc_trimmed/ ../6_alignment .

## JBrowse2

Upload a screenshot of JBrowse2 that shows the reference genome, the annotations, and the two alignments as tracks.

## Snakemake Pipeline (Optional)

1. Copy your `Snakefile` and `config.yaml` from `6_alignment` to `7_alignment_qc`.  
2. In the `Snakefile`, add the following rules:  
    a. `convert_hisat` - use `samtools` to convert HISAT SAMs to BAMs; sort the BAMs.  
    b. `merge_hisat` - use `samtools` to merge all HISAT BAMs.  
    c. `flagstat` -  use `samtools` to generate `flagstat` QC.  
    d. `qualimap` - use `qualimap` to generate `bamqc` and `rnaseq` QC.  
    e. `report` - use `multiqc` to aggregate all QC data into a single report. This should be an update of the report from the previous week's homework.  

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.